Q1

In [33]:
import numpy as np 
import pandas as pd
import warnings

pd.options.display.float_format = '{:.8f}'.format

warnings.filterwarnings("ignore")

def f_a(x):
    return x + np.exp(-x**2)*np.cos(x)

def f_b(x):
    fx = f_a(x)
    return fx*fx

def d_f_a(x):
    return 1 - 2*x*np.exp(-x**2)*np.cos(x) - np.exp(-x**2)*np.sin(x)

def d2_f_a(x):
    return -2*np.exp(-x**2)*np.cos(x) - 4*x*np.exp(-x**2)*np.sin(x) - np.exp(-x**2)*np.cos(x) + 2*x**2*np.exp(-x**2)*np.cos(x) - np.exp(-x**2)*np.sin(x)

def d2_f_b(x):
    return 2*f_a(x)*d2_f_a(x) + 2*d_f_a(x)*d_f_a(x)

def d_f_b(x):
    return 2*f_a(x)*d_f_a(x)


def modified_newtons_method(f:callable, df:callable, d2f:callable, p0: np.float64, tol : np.float64 =1e-6 , max_iter: int = 1000) ->tuple:
    p = p0
    p_values = [p]
    i = 1
    while i < max_iter:
        p = p - f(p)*df(p)/(df(p)**2 - f(p)*d2f(p))
        p_values.append(p)
        if abs((p_values[i] - p_values[i-1])/p_values[i]) < tol:
            break
        i+=1

    return i,p_values

def newton_raphson(f:callable, df:callable,  p0: np.float64, tol : np.float64 =1e-6 , max_iter: int = 1000) ->tuple:
    p = p0
    p_values = [p]
    i = 1
    while i < max_iter:
        p = p - f(p)/df(p)
        p_values.append(p)
        if abs((p_values[i] - p_values[i-1])/p_values[i]) < tol:
            break
        i+=1

    return i,p_values

def secant(f:callable, p0: np.float64, p1: np.float64, tol : np.float64 =1e-6 , max_iter: int = 1000) ->tuple:
    p_values = [p0,p1]
    i = 1
    while i < max_iter: 
        p = p1 - f(p1)*(p1-p0)/(f(p1)-f(p0))
        p_values.append(p)
        if abs((p_values[i] - p_values[i-1])/p_values[i]) < tol:
            break
        p0 = p1
        p1 = p
        i+=1
    return i,p_values

iterations_newton_f_a, p_values_newton_f_a = newton_raphson(f_a, d_f_a, 0)
iterations_newton_f_b, p_values_newton_f_b = newton_raphson(f_b, d_f_b, 0)
iterations_secant_f_a, p_values_secant_f_a = secant(f_a, 0, 1)
iterations_secant_f_b, p_values_secant_f_b = secant(f_b, 0, 1)
iterations_modified_newton_f_a, p_values_modified_newton_f_a = modified_newtons_method(f_a, d_f_a, d2_f_a, 0)
iterations_modified_newton_f_b, p_values_modified_newton_f_b = modified_newtons_method(f_b, d_f_b, d2_f_b, 0)
max_length = max(
    len(p_values_newton_f_a),
    len(p_values_newton_f_b),
    len(p_values_secant_f_a),
    len(p_values_secant_f_b),
    len(p_values_modified_newton_f_a),
    len(p_values_modified_newton_f_b)
    )

p_values_newton_f_a.extend([np.nan] * (max_length - len(p_values_newton_f_a)))
p_values_newton_f_b.extend([np.nan] * (max_length - len(p_values_newton_f_b)))
p_values_secant_f_a.extend([np.nan] * (max_length - len(p_values_secant_f_a)))
p_values_secant_f_b.extend([np.nan] * (max_length - len(p_values_secant_f_b)))
p_values_modified_newton_f_a.extend([np.nan] * (max_length - len(p_values_modified_newton_f_a)))
p_values_modified_newton_f_b.extend([np.nan] * (max_length - len(p_values_modified_newton_f_b)))

data = {
    'Newton a)': p_values_newton_f_a,
    'Secant a)': p_values_secant_f_a,
    'Newton b)': p_values_newton_f_b,
    'Secant b)': p_values_secant_f_b,
    'Modified Newton a)': p_values_modified_newton_f_a,
    'Modified Newton b)': p_values_modified_newton_f_b
}

table = pd.DataFrame(data)
iterations = pd.DataFrame(data =
        {
            'Newton' : [iterations_newton_f_a, iterations_newton_f_b],
            'Secant' : [iterations_secant_f_a, iterations_secant_f_b],
            'Modified Newton' : [iterations_modified_newton_f_a, iterations_modified_newton_f_b]
        }
        ,
        index=['a', 'b']
    )

iterations


,Newton,Secant,Modified Newton
a,5,8,6
b,19,35,6


In [37]:
table

,Newton a),Secant a),Newton b),Secant b),Modified Newton a),Modified Newton b)
0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
1,-1.00000000,1.00000000,-0.50000000,1.00000000,-0.25000000,-0.25000000
2,-0.53064402,-5.03103873,-0.54459804,-2.28811910,-0.49350176,-0.49350176
3,-0.58862653,-0.16051867,-0.56655001,2.23874342,-0.58083250,-0.58083250
4,-0.58840178,-0.82983062,-0.57748280,89.85757285,-0.58835056,-0.58835056
5,-0.58840178,-0.57563757,-0.58294333,2.18452285,-0.58840177,-0.58840177
6,NaN,-0.58871034,-0.58567273,2.13290670,-0.58840178,-0.58840178
7,NaN,-0.58840173,-0.58703729,1.09750165,NaN,NaN
8,NaN,-0.58840178,-0.58771954,0.57216311,NaN,NaN
9,NaN,-0.58840178,-0.58806066,-4.82599571,NaN,NaN
